# CNN model

In [32]:
# Imports

import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import cifar10

### 1. Data Preprocessing

In [ ]:
# Loading CIFAR-10 dataset

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('Training set shape:', x_train.shape)
print('Testing set shape:', x_test.shape)

print('Training labels shape:', y_train.shape)
print('Testing labels shape:', y_test.shape)

labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

print(labels)

In [ ]:
# Visualizing some images

plt.figure(figsize=(8, 8))

for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_train[i])
    plt.xlabel(labels[y_train[i][0]])

plt.show()

In [ ]:
# Normalizing the Images

x_train = x_train / 255
x_test = x_test / 255

print(x_train[0].min())
print(x_train[0].max())

In [ ]:
# Resizing images

x_train = tf.image.resize(x_train, (64, 64))
x_test = tf.image.resize(x_test, (64, 64))

print('Resized training image size:', x_train[0].shape)
print('Resized testing image size:', x_test[0].shape)

In [37]:
# Augmenting data

# TODO